In [1]:
import random
from random import shuffle

from PIL import Image, ImageFilter
from IPython import display

from matplotlib.pyplot import imshow
%matplotlib inline

import pickle
import time
import error_table

from sampling_utils import *
#from gen_utils import *
#from evaluation_utils import *
#from config_db import ConfigDB

In [2]:
num_pics = 500
min_num_cars = 1
max_num_cars = 4

domains = [[52,87],[1,37],[0,1],[0.35,1],[0.5,1.5],[0.25,1],[0.5,1],[0.5,1]]
types = ['int','int','float','float','float','float','float','float']

gen_misclass = True
store_config_db = False
store_err_tab = True

IOMIN_THRESH = 0.4
IOU_THRESH = 0.5
AP_THRESH = 0.75
REC_THRESH = 0.75
NN_PROB_THRESH = 0.5

if gen_misclass:
    checkpoint = '/home/tommaso/analyzeNN/data/train_0/checkpoint/train/model.ckpt-5000'
    import squeezedet as nn
    net = nn.init(checkpoint)

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
for j in range(min_num_cars,max_num_cars):
    
    total_attempts = 0
    rejections = 0
    
    # Init data structure
    if store_err_tab:
        err_table = error_table()
    if store_config_db:
        config_db = ConfigDB()
    
    path_data_set = './data/mis_halt/'
    #path_data_set = '/home/tommaso/analyzeNN/data/train_0/mis/mis_' + str(j) + '/'
    
    i = 0
    v = 0

    while i < num_pics:
        
        v = v+1

        # Sample configuration
        #num_cars = random.randint(min_num_cars, max_num_cars)
        num_cars = 3
        sample_dims = [1,num_cars,num_cars,num_cars,1,1,1,1]
        sample, norm_sample = get_sample(v, 'random', types=types, domains=domains, dims=sample_dims)
        
        print sample
        sample[3].sort(reverse=True)
        norm_sample[3].sort(reverse=True)
        
        total_attempts += 1
        
        good_sample = True
        # Check sample distance from stored ones
        if store_config_db:
            sample_pad = pad_sample(sample)
            good_sample = config_db.eps_dist(sample_pad,0.25)
            
            
        if good_sample:
            # Generate image from sample
            img, ground_boxes, img_info = gen_image(sample)

            # Is it a good picture? (cars don't overlap too much)
            save_img = True
            for m in range(len(ground_boxes)):
                for n in range(m+1, len(ground_boxes)):
                    if iomin(ground_boxes[m], ground_boxes[n]) > IOMIN_THRESH:
                        save_img = False

            ap = 0
            recall = 0
                        
            # Check misclassification
            if gen_misclass and save_img:
                # Save tmp image    
                tmp_img_file_name = './tmp.png'
                img.save(tmp_img_file_name)

                # Classify img
                (boxes,probs,cats) = nn.classify(tmp_img_file_name, net, NN_PROB_THRESH) 
                print boxes

                # Not cars
                if any(cats):
                    save_img = True
                else:
                    ap, recall = average_precision(ground_boxes, boxes, IOU_THRESH)
                    save_img =  (ap <= AP_THRESH or recall <= REC_THRESH)
                        
                        

            if save_img:
                
                if store_config_db:
                    config_db.add(sample_pad)            
                
                file_name = 'm_h_' + str(j) + '_' + str(i).zfill(6)
                #file_name = 'm_d_' + str(i).zfill(6)
                
                print(file_name + ' ' + str(ap) + ' ' + str(recall))
                
                # Save data
                save_image(img, file_name, path_data_set)
                save_label(ground_boxes, file_name, path_data_set)
                
                # Save in image set
                with open(path_data_set + 'image_set_m_1.txt', "a+") as im_set:
                    im_set.write(file_name + '\n')
                    im_set.close()
                
                if store_err_tab:
                    err_table.update_with_elem(img_info)
                    
                i+=1
        else:
            rejections += 1

#         if(total_attempts %50 == 0):
#             print('gen pics: ' + str(total_attempts) + ' | misclass pics: ' + str(i) + ' | rejections: ' +str(rejections))
            
print('END gen pics: ' + str(total_attempts) + ' | misclass pics: ' + str(i) + ' | rejections: ' +str(rejections))

if store_err_tab:
    err_table_file = open(path_data_set + 'error_table' + str(),'w')
    pickle.dump(err_table,err_table_file)
    err_table_file.close()

[[81], [5, 22, 3], [0.20290874680020143, 0.42635901241204543, 0.03128824174179101], [0.7699467020911523, 0.9818600012754808, 0.9532591697655478], [0.5393108094576002], [0.540711638646866], [0.9603979943311902], [0.7794324541236362]]
[[56], [18, 31, 14], [0.6676829455131602, 0.10368826290655553, 0.44666692237126104], [0.6973363435773883, 0.5310056779723238, 0.8692341915646726], [0.8229227071207306], [0.6659448881001704], [0.750826635043378], [0.8980703224706716]]
[[70], [21, 36, 30], [0.25344217996045737, 0.01304580129225863, 0.4458595533553632], [0.4876184482043513, 0.7437395078562044, 0.5077895346896829], [1.3107980568160644], [0.8175204715937628], [0.9159845665510438], [0.995691861784769]]
[[81], [12, 11, 23], [0.5707989304684811, 0.8371144878412045, 0.45283347821661424], [0.7417575658743389, 0.5635411754572783, 0.9263827488773838], [0.9578983998123671], [0.6338961408219675], [0.8029740862300849], [0.5672364336]]
[[86], [27, 34, 10], [0.8435376395697838, 0.5473000319890183, 0.0777426

KeyboardInterrupt: 

In [52]:
(boxes,probs,cats) = nn.classify('./data/train_0/mis/images/m_0_3_000049.png', net, NN_PROB_THRESH) 

In [53]:
labels = read_label('./data/train_0/mis/labels/m_0_3_000049.txt')
label_boxes = [ kitti_2_box_format(l) for l in labels ]
ap, recall = average_precision(label_boxes, boxes, IOU_THRESH)

In [54]:
ap

0.75

In [55]:
recall

1.0

In [38]:
boxes

[array([ 584.81933594,  220.34779358,  129.04016113,  105.63809204], dtype=float32),
 array([ 589.07788086,  173.05867004,   62.38708496,   49.52755737], dtype=float32),
 array([ 635.69897461,  169.13327026,   56.59399414,   41.10015869], dtype=float32)]

In [42]:
gb

[[610.0, 145.0, 654.0, 177.0],
 [556.0, 144.0, 612.0, 192.0],
 [518.0, 179.0, 646.0, 265.0]]

In [56]:
net.close()

AttributeError: 'tuple' object has no attribute 'close'

In [5]:
e = pickle.load( open( "error_table", "rb" ) )

In [6]:
type(e)

instance

In [13]:
e.entries.keys()

['foreground0_width',
 'color_sample',
 'road_path',
 'environment',
 'road_id',
 'foreground0car_color',
 'foreground0car_orientation',
 'foreground0_x',
 'foreground0car_category',
 'road_type',
 'foreground0car_id',
 'foreground0type',
 'sharpness_sample',
 'contrast_sample',
 'foreground0car_path',
 'foreground0_height',
 'background_color',
 'foreground0_y',
 'brightness_sample']

In [3]:
num_cars=1
sample_dims = [1,num_cars,num_cars,num_cars,1,1,1,1]
sample, norm_sample = get_sample(0, 'halton', types=types, domains=domains, dims=sample_dims)

IndexError: list index out of range